# LELA32052 Computational Linguistics Week 3

In [ ]:
import re

### Escaping special characters
We have learned about a number of characters that have a special meaning in regular expressions (periods, dollar signs etc). We might sometimes want to search for these characters in strings. To do this we can "escape" the character using a double backslash() as follows:


In [ ]:
opening_sentence = "On an exceptionally hot evening early in July a young man came out of the garret in which he lodged in S. Place and walked slowly, as though in hesitation, towards K. bridge."
re.findall("\\.",opening_sentence)

### re.split()
In week 1 we learned to tokenise a string using the string function split. re also has a split function. re.split() takes a regular expression as a first argument (unless you have a precompiled pattern) and a string as second argument, and split the string into tokens divided by all substrings matched by the regular expression.


**Problem 1: Can you improve on the following tokeniser?**

In doing so you might need to extend your knowledge of regular expressions and employ one of the special characters included here: https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf


In [ ]:
to_split_on_word = re.compile(" ")
opening_sentence_new = to_split_on_word.split(opening_sentence)
print(opening_sentence_new)

# Sentence Segmentation

Above we split a sentence into words. However most texts that we want to process have more than one sentence, so we also need to segment text into sentences. We will work with the first chapter of Crime and Punishment again

In [ ]:
from io import RawIOBase
!wget https://www.gutenberg.org/files/2554/2554-0.txt
f = open('2554-0.txt')
raw= f.read()
chapter_one = raw[5464:23725]
chapter_one = re.sub('\n',' ',chapter_one)

Just as for segmenting sentences into words, we can segment texts into sentence using the re.split function. If you run the code below you will get a list of words.

**Problem 2: What regular expression could we use to split a text into a list of sentences?**

In [ ]:
to_split_on_sent = re.compile(" ")
C_and_P_sentences = to_split_on_sent.split(chapter_one)
print(C_and_P_sentences)

### For Loops

A commonly used tool in programming is the "for loop". In its simplest form this allows us to iterate over (move through) a series of values. For example:

In [ ]:
for i in range(0,10):
  print(i)

It can also be used to iterate through data structures:


In [ ]:
sentence=["the","boy","went","to","the","park"]
for word in sentence:
  print(word)

**Problem 3: How might we use a for loop plus what you learned above in order to turn an input text into list of sentences, where each sentence is a list of words?**

# Vector semantics

In this week's lecture you heard about Vector-based semantics. Today we will take a look at these models in Python.

First we will use the method we arrived at above to segment and tokenize the whole of Crime and Punishment.

In [ ]:
to_split_on_sent = re.compile("\\. ")
to_split_on_words = re.compile(" \\.|, | ")
C_and_P_tokens_sentences=[]

C_and_P_sentences = to_split_on_sent.split(raw)
for sentence in C_and_P_sentences:
    C_and_P_tokens_sentences.append(to_split_on_words.split(sentence))

Next we will build a cooccurence matrix using the following code. The purpose of this is to aid your conceptual understanding by looking at the output, and you aren't expected to read or understand this code. Although if you come back to it later in the semester you may well be able to figure it out

In [ ]:
import numpy as np
c_and_p=C_and_P_tokens_sentences
c_and_p = [x for l in c_and_p for x in l]
token_count = len(c_and_p)
type_list = list(set(c_and_p))
# The type count is the number of unique words. The token count is the total number of words including repetitions.
type_count = len(type_list)
# We create a matrix in which to store the counts for each word-by-word co-occurence
M = np.zeros((type_count, type_count))
window_size = 2

for i, word in enumerate(c_and_p):
            #print(str(i) + word)
            # Find the index in the tokenized sentence vector for the beginning of the window (the current token minus window size or zero whichever is the lower)
            begin = max(i - window_size, 0)
            # Find the index in the tokenized sentence vector for the end of the window (the current token plus window size or the length of the sentence whichever is the lower)
            end  = min(i + window_size, token_count)
            # Extract the text from beginning of window to the end
            context = c_and_p[begin: end + 1]
            # Remove the target word from its own window
            context.remove(c_and_p[i])
            # Find the row for the current target word
            current_row = type_list.index(c_and_p[i])
            # Iterate over the window for this target word
            for token in context:
                # Find the ID and hence the column index for the current token
                current_col = type_list.index(token)
                # Add 1 to the current context word dimension for the current target word
                M[current_row, current_col] += 1

In [ ]:
def cosine(a,b):
  return(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)))

In [ ]:
import numpy as np
w1 = "walk"
w2 = "run"
w3 = "shine"
w1_index = type_list.index(w1)
w2_index = type_list.index(w2)
w3_index = type_list.index(w3)
w1_vec=M[type_list.index(w1),]
w2_vec=M[type_list.index(w2),]
w3_vec=M[type_list.index(w3),]


In [ ]:
cosine(w1_vec,w2_vec)

### Pretrained embeddings

Vectors are best when learned from very large text collections. However learning such vectors, particular using neural network methods rather than simple counting, is very computationally intensive. As a result most people make use of pretrained embeddings such as those found at

https://code.google.com/archive/p/word2vec/

or

https://nlp.stanford.edu/projects/glove/

In [ ]:
!gdown 1Km0l6ekgFbpB5VS5TrxQDtyRon5eQVvL
!unzip glove.6B.100d.txt.zip

In [ ]:
import numpy as np
embedding_file = 'glove.6B.100d.txt'
#embedding_file = f.read()
embeddings=[]
type_list=[]
with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                type_list.append(word)
                embeddings.append(vec)
M=np.array((embeddings))

In [ ]:
w1 = "football"
w2 = "rugby"
w3 = "cricket"
w1_index = type_list.index(w1)
w2_index = type_list.index(w2)
w3_index = type_list.index(w3)
w1_vec=M[w1_index,]
w2_vec=M[w2_index,]
w3_vec=M[w3_index,]

In [ ]:
cosine(w1_vec,w2_vec)

In [ ]:
cosine(w1_vec,w3_vec)

In [ ]:
cosine(w2_vec,w3_vec)

**Problem 4. Calculate the cosine between the words above. What do the cosine values tell us?**

# Finding the most similar words

One thing we often want to do is to find the most similar words to a given word/vector. An exhaustive N x N comparison is very time consuming, and so we can make use of an efficient "nearest neighbours" finding algorithm. We are just using this algorithm here so we won't go into it in any detail.

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(M)

In [ ]:
w="football"
w_index = type_list.index(w)
w_vec = M[w_index,]
for i in nbrs.kneighbors([w_vec])[1][0]:
  print(type_list[i])

**Problem 5. Find some examples where the system fails and explain why you think it has done so.**

### Analogical reasoning

Another semantic property of embeddings is their ability to capture relational meanings. In an important early vector space model of cognition, Rumelhart and Abrahamson (1973) proposed the parallelogram model for solving simple analogy problems of the form a is to b as a* is to what?. In such problems, a system given a problem like apple:tree::grape:?, i.e., apple is to tree as  grape is to , and must fill in the word vine.

In the parallelogram model, the vector from the word apple to the word tree (= tree − apple) is added to the vector for grape (grape); the nearest word to that point is returned.





In [ ]:
w1 = "apple"
w2 = "tree"
w3 = "grape"
w1_index = type_list.index(w1)
w2_index = type_list.index(w2)
w3_index = type_list.index(w3)
w1_vec = M[w1_index,]
w2_vec = M[w2_index,]
w3_vec = M[w3_index,]

spatial_relationship = w2_vec - w1_vec
w4_vec = w3_vec + spatial_relationship

nbrs.kneighbors([w4_vec])
for i in nbrs.kneighbors([w4_vec])[1][0]:
  print(type_list[i])

**Problem 6: Come up with analogical reasoning problems of your own and use the code to solve it. Try to find one for which the approach works and one for which it doesn't work.**